In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
trainlist = os.listdir("../input/train")
testlist = os.listdir("../input/test")
df_sample = pd.read_csv('../input/sample_submission.csv')
df_labels = pd.read_csv('../input/labels.csv')

Printing the first 5 rows of each dataframes :

In [3]:
df_sample.head()

In [4]:
df_labels.head()

In [5]:
df_labels.describe()

In [ ]:
import seaborn as sns
sns.countplot(x='breed',data = df_labels)

The groups look balanced enough. We proceed to preprocessing the data. First we set the Y, or label data as one hot vectors.

In [ ]:
from keras.utils import to_categorical
from numpy import array
breeds = df_labels['breed'].tolist()
breed = df_labels['breed'].unique()
unq_to_breed = dict()
for i in range(len(df_labels)):
    unq_to_breed[str(df_labels['id'][i])]=(df_labels['breed'][i])
breed_to_id = dict((j,i) for i,j in enumerate(breed))
id_list = to_categorical(list(breed_to_id.values())).astype(np.float32)
c=0
for i in unq_to_breed:
    print(i,":",unq_to_breed[i])
    c+=1
    if c==10:
        break

In [ ]:
c=0
for i in breed_to_id:
    print(i,":",breed_to_id[i])
    c+=1
    if c==10:
        break

In [ ]:
#we have unq to breed and breed to id so now we create unq to id ! 
unq_to_id = dict()
for i in unq_to_breed : 
    unq_to_id[i] = id_list[breed_to_id[unq_to_breed[i]]]
c=0
for i in  unq_to_id:
    print(i,":", unq_to_id[i])
    c+=1
    if c==5:
        break

As Unq_to_id is our final preprocessing stuff, let's see what it actually is :
---

In [ ]:
print("length : ",len(unq_to_id))
print("First element :",unq_to_id["000bec180eb18c7604dcecc8fe0dba07"])
print("Element length : ",len(unq_to_id["000bec180eb18c7604dcecc8fe0dba07"]))
print("\nLets see if all's well\n")
print("Checking for 0f04466edd10d6c1d27e123399cf4433")
print("unq to breed : ",unq_to_breed["0f04466edd10d6c1d27e123399cf4433"])
print("breed to id : ",breed_to_id[unq_to_breed["0f04466edd10d6c1d27e123399cf4433"]])
print("unq to id : ",unq_to_id["0f04466edd10d6c1d27e123399cf4433"])

Thus we see that the unq_to_id ( unique to id ) dict contains a mapping from id to one hot encoded vectors of length 120. All's cool till now.
---

The Y part is done. Now off to preprocessing the images as X.

In [ ]:
#Plotting a random image
import cv2
import matplotlib.pyplot as plt
image = cv2.imread("../input/train/0f04466edd10d6c1d27e123399cf4433.jpg")
%matplotlib inline
imgplot = plt.imshow(image)

In [ ]:
#storing all images in a variable X
X = []
Y = []
for i in range(len(trainlist)):
    image = cv2.imread("../input/train/"+trainlist[i])
    image = cv2.resize(image,(120,120), interpolation = cv2.INTER_CUBIC)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    X.append(image)
    Y.append(unq_to_id[trainlist[i][0:len(trainlist[i])-4]])
print("Preprocessing done")    

In [ ]:
X=np.array(X)
X=X.reshape(len(X),120,120,1)
X.shape[1:]

In [ ]:
X.shape

In [ ]:
Y=np.array(Y)
class_totals = Y.sum(axis=0)
class_weight = class_totals.max() / class_totals
Y

In [ ]:
print("X length : ",len(X))
print("Y length : ",len(Y))
print("X.shape : ", X.shape)
print("Y.shape : ", Y.shape)

Now we build our model using keras ! 
---

In [ ]:
from keras.utils import np_utils
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
nb_filters = 32
nb_classes = 120
nb_pool = 2
nb_conv = 3

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(nb_classes, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
validation_split = 0.10
model.fit(X, Y, batch_size=20, class_weight=class_weight, epochs=10, verbose=1, validation_split=validation_split)

As expected the accuracy is low as we are not using pretrained weights and training the model on local system itself. Let's try with VGG19 model next : 

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
vgg = VGG19(#weights='imagenet',
    weights = None, include_top=False, input_shape=(120, 120, 1))
x = vgg.output
x = Flatten()(x)
predictions = Dense(nb_classes, activation='softmax')(x)
vggmodel = Model(inputs=vgg.input, outputs=predictions)
for layer in vgg.layers:
    layer.trainable = False

vggmodel.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
vggmodel.summary()

In [ ]:
validation_split = 0.10
vggmodel.fit(X, np.array(Y), batch_size=20, class_weight=class_weight, epochs=10, verbose=1, validation_split=validation_split)

As we see, we don't get good accuracy as we don't use fully trained weights.